In [ ]:
from google.colab import drive
drive.mount("/content/drive",force_remount=True)

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# ONLY TO RUN FOR CLEMENT (MY PATH)
RawDataB_weather = pd.read_csv("/content/drive/MyDrive/CS_project/Data/RawDataB_weather.csv")
RawDataB_weather.head()

### Date Handling

In [ ]:
# 1) DATE

#create column date (eg. 7/21/2019)
#create column weekday (eg. Sunday)
#create column month (eg. 7)
#create colum year (eg. 2019)

In [ ]:
RawDataB_weather = RawDataB_weather.rename(columns={"Date": "old_date"})

In [ ]:
RawDataB_weather["date"] = pd.to_datetime(RawDataB_weather["old_date"], format="%a %d/%m/%y", errors="coerce") # there are some unknowns so we need coerce

# Changed the format to "%a %m/%d/%y" to include the weekday abbreviation
# "%a" represents the abbreviated weekday name (e.g., Sun, Mon, Tue)

RawDataB_weather["Date"] = RawDataB_weather["date"].dt.strftime("%m-%d-%Y")
RawDataB_weather["Weekday"] = RawDataB_weather["date"].dt.day_name()
RawDataB_weather["Month"] = RawDataB_weather["date"].dt.month
RawDataB_weather["Year"] = RawDataB_weather["date"].dt.year

In [ ]:
RawDataB_weather

### Add Season

In [ ]:
conditions = [
    RawDataB_weather["Year"] == 2019,
    (RawDataB_weather["Year"] == 2020) & (RawDataB_weather["Month"] < 7),
    (RawDataB_weather["Year"] == 2020) & (RawDataB_weather["Month"] >= 7),
    (RawDataB_weather["Year"] == 2021) & (RawDataB_weather["Month"] < 7),
    (RawDataB_weather["Year"] == 2021) & (RawDataB_weather["Month"] >= 7),
    (RawDataB_weather["Year"] == 2022) & (RawDataB_weather["Month"] < 7),
    (RawDataB_weather["Year"] == 2022) & (RawDataB_weather["Month"] >= 7),
    (RawDataB_weather["Year"] == 2023) & (RawDataB_weather["Month"] < 7),
    (RawDataB_weather["Year"] == 2023) & (RawDataB_weather["Month"] >= 7),
    (RawDataB_weather["Year"] == 2024) & (RawDataB_weather["Month"] < 7),
    (RawDataB_weather["Year"] == 2024) & (RawDataB_weather["Month"] >= 7)
]

choices = [
    "2019/2020",
    "2019/2020",
    "2020/2021",
    "2020/2021",
    "2021/2022",
    "2021/2022",
    "2022/2023",
    "2022/2023",
    "2023/2024",
    "2023/2024",
    "2024/2025"
]

RawDataB_weather["Season"] = np.select(conditions, choices, default="Unknown")

In [ ]:
RawDataB_weather.head()

### Add Quarters

In [ ]:
import numpy as np

RawDataB_weather["quarter"] = np.where((RawDataB_weather["Month"] >= 1) & (RawDataB_weather["Month"] <= 3), 1,
                             np.where((RawDataB_weather["Month"] >= 4) & (RawDataB_weather["Month"] <= 6), 2,
                                      np.where((RawDataB_weather["Month"] >= 7) & (RawDataB_weather["Month"] <= 9), 3,
                                               np.where((RawDataB_weather["Month"] >= 10) & (RawDataB_weather["Month"] <= 12), 4, np.nan))))

In [ ]:
RawDataB_weather

### Time Handling

In [ ]:
RawDataB_weather["Time"] = pd.to_datetime(RawDataB_weather["Time"],format="%I:%M %p",errors="coerce").dt.hour

### Drop Covid

In [ ]:
# source drop of restriction: https://de.wikipedia.org/wiki/Division_1A_2021/22
# source beginning of restrictions: https://de.wikipedia.org/wiki/COVID-19-Pandemie_in_Belgien

In [ ]:
# Months of Covid
months_to_remove_2020 = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
months_to_remove_2021 = [1, 2, 3, 4, 5, 6, 7, 8]

# Remove rows for the period from March 2020 to May 2021
RawDataB_weather = RawDataB_weather[~(((RawDataB_weather["Year"] == 2020) & (RawDataB_weather["Month"].isin(months_to_remove_2020))) |
          ((RawDataB_weather["Year"] == 2021) & (RawDataB_weather["Month"].isin(months_to_remove_2021))))]

In [ ]:
RawDataB_weather.head()

In [ ]:
unique_home_teams = RawDataB_weather["Home Team"].unique()
print(unique_home_teams)

### Adding Stadium and Location Informations

Club Brugge	https://en.wikipedia.org/wiki/Club_Brugge_KV

Cercle Brugge	https://en.wikipedia.org/wiki/Cercle_Brugge_K.S.V

Genk	https://en.wikipedia.org/wiki/K.R.C._Genk

RSC Anderlecht	https://en.wikipedia.org/wiki/R.S.C._Anderlecht

Union SG	https://en.wikipedia.org/wiki/Royale_Union_Saint-Gilloise

KAA Gent	https://en.wikipedia.org/wiki/K.A.A._Gent

Royal Antwerp	https://en.wikipedia.org/wiki/Royal_Antwerp_F.C

KVC Westerlo	https://en.wikipedia.org/wiki/K.V.C._Westerlo

Standard Liège	https://en.wikipedia.org/wiki/Standard_Li%C3%A8ge

KV Mechelen	https://en.wikipedia.org/wiki/K.V._Mechelen

R Charleroi SC	https://en.wikipedia.org/wiki/R._Charleroi_S.C

OH Leuven	https://en.wikipedia.org/wiki/Oud-Heverlee_Leuven

Sint-Truiden	https://en.wikipedia.org/wiki/Sint-Truidense_V.V

FCV Dender EH	https://en.wikipedia.org/wiki/F.C.V._Dender_E.H

Zulte Waregem	https://en.wikipedia.org/wiki/S.V._Zulte_Waregem

La Louvière	https://en.wikipedia.org/wiki/R.A.A._Louvi%C3%A9roise

In [ ]:
# 🇧🇪 Stadium information for Belgian Pro League clubs (2024–25)
stadium_info = {
    "Club Brugge": ["Jan Breydel Stadion", 29062, "Bruges", "VWV", 0],
    "Cercle Brugge": ["Jan Breydel Stadion", 29062, "Bruges", "VWV", 0],
    "Genk": ["Cegeka Arena", 23500, "Genk", "LIM", 1],
    "RSC Anderlecht": ["Lotto Park", 22500, "Anderlecht (Brussels)", "BRU", 1],
    "Union SG": ["Joseph Marien Stadium", 9400, "Forest (Brussels)", "BRU", 0],
    "KAA Gent": ["Ghelamco Arena", 20000, "Ghent", "VOV", 1],
    "Royal Antwerp": ["Bosuilstadion", 16644, "Antwerp", "ANT", 0],
    "KVC Westerlo": ["Het Kuipje", 8000, "Westerlo", "ANT", 0],
    "Standard Liège": ["Stade Maurice Dufrasne", 27670, "Liège", "LIE", 0],
    "KV Mechelen": ["AFAS Stadion", 16500, "Mechelen", "ANT", 0],
    "R Charleroi SC": ["Stade du Pays de Charleroi", 15000, "Charleroi", "WHT", 0],
    "OH Leuven": ["Den Dreef", 10500, "Leuven", "VBR", 1],
    "Sint-Truiden": ["Stayen", 14600, "Sint-Truiden", "LIM", 1],
    "FCV Dender EH": ["Van Roystadion", 6200, "Denderleeuw", "VBR", 0],
    "Zulte Waregem": ["Regenboogstadion", 12400, "Waregem", "WVL", 0],
    "La Louvière": ["Stade du Tivoli", 12000, "La Louvière", "WHT", 0]
}

# Apply the mapping to your DataFrame
for team, stadium_data in stadium_info.items():
    RawDataB_weather.loc[
        RawDataB_weather["Home Team"] == team,
        ["Stadium", "Max Capacity", "City", "Province", "Full Roof"]
    ] = stadium_data


In [ ]:
RawDataB_weather

### Remove Games not in Juplier League Stadium

In [ ]:
# List of teams to keep
teams_to_keep = [
    "Club Brugge", "Cercle Brugge", "Genk", "RSC Anderlecht", "Union SG", "KAA Gent", "Royal Antwerp", "KVC Westerlo", "Standard Liège", "KV Mechelen", "R Charleroi SC", "OH Leuven", "Sint-Truiden", "FCV Dender EH", "Zulte Waregem", "La Louvière"
]

# Keep only rows where the "Home Team" is in the list of specified teams
RawDataB_weather = RawDataB_weather[RawDataB_weather["Home Team"].isin(teams_to_keep)]

In [ ]:
RawDataB_weather = RawDataB_weather[RawDataB_weather["Competition"] == "Jupiler Pro League"]

### Remove Games not Yet played

In [ ]:
# Remove rows where the "Result" column has the value "-:-"
RawDataB_weather = RawDataB_weather[RawDataB_weather["Result"] != "-:-"]

 ### Handling Stadium Attendance

In [ ]:
RawDataB_weather["Attendance"]

In [ ]:
#transform 9.444 into 9444
RawDataB_weather["Attendance"] = RawDataB_weather["Attendance"].astype(str).str.replace(".", "")

In [ ]:
RawDataB_weather = RawDataB_weather[(RawDataB_weather["Attendance"] != "x") & (RawDataB_weather["Attendance"] != "nan")]

In [ ]:
output_csv = "/content/drive/My Drive/CS_project/Data/Cleaned_RawDataB.csv"
RawDataB_weather.to_csv(output_csv, index=False)

### Add Attendance in Percentage

In [ ]:
# Convert "Attendance" and "Max Capacity" columns to numeric, handling any errors
RawDataB_weather["Attendance"] = pd.to_numeric(RawDataB_weather["Attendance"], errors="coerce")
RawDataB_weather["Max Capacity"] = pd.to_numeric(RawDataB_weather["Max Capacity"], errors="coerce")

# Remove rows with NaN values in "Attendance" or "Max Capacity" after conversion
RawDataB_weather.dropna(subset=["Attendance", "Max Capacity"], inplace=True)

# Calculate the "PercentageAttendance" column
RawDataB_weather["PercentageAttendance"] = RawDataB_weather["Attendance"] / RawDataB_weather["Max Capacity"]

In [ ]:
count = RawDataB_weather[RawDataB_weather["PercentageAttendance"] == 1].shape[0]
print(f"Number of occurrences where PercentageAttendecance is 1: {count}")

In [ ]:
count = RawDataB_weather[RawDataB_weather["PercentageAttendance"] > 1].shape[0]
print(f"Number of occurrences where PercentageAttendecance is over 1: {count}")
# Standing Areas: Some stadiums have areas where people can stand rather than sit, allowing them to accommodate more people than their official seating capacity.
# Temporary Seating: During special events or high-profile games, temporary seating may be added to increase capacity.

### Handling Rankings

In [ ]:
RawDataB_weather["Ranking Home Team"] = RawDataB_weather["Ranking Home Team"].str.replace(r"[().]", "", regex=True).str.strip()
RawDataB_weather["Ranking Away Team"] = RawDataB_weather["Ranking Away Team"].str.replace(r"[().]", "", regex=True).str.strip()

In [ ]:
RawDataB_weather.head()

### Remove Doubles

In [ ]:
# Create a unique match identifier by including the competition, season, sorted home/away teams, and month
RawDataB_weather["Match_ID"] = RawDataB_weather.apply(lambda row: f"{row["Competition"]}-{row["Season"]}-{row["Month"]}-{"-".join(sorted([row["Home Team"], row["Away Team"]]))}", axis=1)

# Drop duplicate games based on the generated "Match_ID"
RawDataB_weather = RawDataB_weather.drop_duplicates(subset="Match_ID")

# Drop the "Match_ID" column if it"s no longer needed
RawDataB_weather = RawDataB_weather.drop(columns=["Match_ID"])

### Add GDP lagged

In [ ]:
import pandas as pd
import requests
import io
from pathlib import Path

# ------------------------------
# Helper: Fetch from FRED
# ------------------------------
def fetch_fred_series(series_id, col_name):
    """Download a FRED series by ID and rename its value column."""
    try:
        url = f"https://fred.stlouisfed.org/graph/fredgraph.csv?id={series_id}"
        r = requests.get(url, timeout=10)
        r.raise_for_status()
        df = pd.read_csv(io.StringIO(r.text))

        # ✅ FIXED: added "observation_date"
        date_col = None
        for col in ["DATE", "date", "Date", "observation_date"]:
            if col in df.columns:
                date_col = col
                break

        if date_col is None:
            print(f"⚠️ No date column found in {series_id}. Columns: {list(df.columns)}")
            return pd.DataFrame(columns=["Date", col_name])

        value_col = None
        for col in df.columns:
            if col != date_col:
                value_col = col
                break

        if value_col is None:
            print(f"⚠️ No value column found in {series_id}")
            return pd.DataFrame(columns=["Date", col_name])

        df[date_col] = pd.to_datetime(df[date_col])
        df = df.rename(columns={date_col: "Date", value_col: col_name})
        df = df[["Date", col_name]]
        df[col_name] = pd.to_numeric(df[col_name], errors="coerce")

        print(f"✅ Fetched {series_id}: {len(df)} rows")
        return df

    except Exception as e:
        print(f"⚠️ Error fetching {series_id}: {e}")
        import traceback
        traceback.print_exc()
        return pd.DataFrame(columns=["Date", col_name])


# ------------------------------
# Helper: Fetch from World Bank (annual)
# ------------------------------
def fetch_wb_series(country, indicator, col_name, start=2015):
    try:
        url = f"https://api.worldbank.org/v2/country/{country}/indicator/{indicator}?date={start}:2025&format=json&per_page=500"
        r = requests.get(url, timeout=10)
        r.raise_for_status()
        data = r.json()
        if len(data) < 2 or not data[1]:
            print(f"⚠️ No data returned for {indicator}")
            return pd.DataFrame(columns=["Date", col_name])
        df = pd.DataFrame(data[1])[["date", "value"]]
        df = df.rename(columns={"date": "Date", "value": col_name})
        df["Date"] = pd.to_datetime(df["Date"], format="%Y")
        df = df.dropna().sort_values("Date")
        print(f"✅ Fetched {indicator}: {len(df)} rows")
        return df
    except Exception as e:
        print(f"⚠️ Error fetching {indicator}: {e}")
        return pd.DataFrame(columns=["Date", col_name])


# ------------------------------
# 1️⃣ Quarterly data (FRED)
# ------------------------------
print("📊 Fetching quarterly data from FRED...")
gdp = fetch_fred_series("CLVMNACSCAB1GQBE", "GDP_Real")
cpi = fetch_fred_series("CPALTT01BEQ657N", "CPI_QoQ_Growth_%")
emp = fetch_fred_series("LREM64TTBEQ156N", "Employment_Rate_%")

qdata = gdp.merge(cpi, on="Date", how="outer").merge(emp, on="Date", how="outer")
qdata = qdata[(qdata["Date"] >= "2018-01-01") & (qdata["Date"] <= "2025-12-31")]
qdata = qdata.sort_values("Date").reset_index(drop=True)

if qdata.empty:
    print("⚠️ Warning: No quarterly data available after filtering!")
else:
    print(f"✅ Quarterly data prepared: {len(qdata)} rows")

# ------------------------------
# 2️⃣ Annual data (World Bank)
# ------------------------------
print("\n📊 Fetching annual data from World Bank...")
gdp_nom = fetch_wb_series("BE", "NY.GDP.MKTP.CD", "GDP_Nominal_USD")
infl = fetch_wb_series("BE", "FP.CPI.TOTL.ZG", "CPI_Annual_%")
unemp = fetch_wb_series("BE", "SL.UEM.TOTL.ZS", "Unemployment_%")

adata = gdp_nom.merge(infl, on="Date", how="outer").merge(unemp, on="Date", how="outer")
adata = adata[(adata["Date"] >= "2018-01-01") & (adata["Date"] <= "2025-12-31")].reset_index(drop=True)

if adata.empty:
    print("⚠️ Warning: No annual data available after filtering!")
else:
    print(f"✅ Annual data prepared: {len(adata)} rows")

# ------------------------------
# 3️⃣ Combine + Save
# ------------------------------
output_file = "belgium_economic_data.xlsx"
with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    qdata.to_excel(writer, sheet_name="Quarterly_Data", index=False)
    adata.to_excel(writer, sheet_name="Annual_Data", index=False)

print(f'\n✅ Belgium Economic Data (2018–2025) saved as "{output_file}"')
print(f"📁 Full path: {Path(output_file).absolute()}")

print("\n📊 Summary:")
print(f"Quarterly data: {len(qdata)} rows | Variables: {list(qdata.columns)}")
print(f"Annual  data: {len(adata)} rows | Variables: {list(adata.columns)}")


In [ ]:
# --- Quarter-by-quarter macro merge (prev available quarter), no _x/_y leftovers ---

import re

# Helper: squash any *_x / *_y duplicates into a single base column and drop the rest
def squash_suffix_pairs(df: pd.DataFrame) -> pd.DataFrame:
    bases = {re.sub(r'_(x|y)$', '', c) for c in df.columns if re.search(r'_(x|y)$', c)}
    for base in bases:
        candidates = [base, f"{base}_x", f"{base}_y"]
        s = None
        for col in candidates:
            if col in df.columns:
                s = df[col] if s is None else s.combine_first(df[col])
        if s is not None:
            df[base] = s
        df.drop(columns=[c for c in (f"{base}_x", f"{base}_y") if c in df.columns],
                inplace=True, errors="ignore")
    return df

# 0) Make sure football df has Year/quarter as integers
RawDataB_weather["Year"]    = pd.to_numeric(RawDataB_weather["Year"], errors="coerce").astype("Int64")
RawDataB_weather["quarter"] = pd.to_numeric(RawDataB_weather["quarter"], errors="coerce").astype("Int64")

# 1) Build (Year, Quarter) macro table from qdata
macro = qdata.copy()
macro["Year"]    = macro["Date"].dt.year
macro["Quarter"] = macro["Date"].dt.quarter

# If multiple rows per quarter exist, keep the last chronological value
macro_q = (
    macro.sort_values("Date")
         .groupby(["Year", "Quarter"], as_index=False)
         .last()
)

macro_cols = [c for c in macro_q.columns if c not in ["Date", "Year", "Quarter"]]

# 2) Create a complete year–quarter grid and left-join macro
min_year, max_year = int(macro_q["Year"].min()), int(macro_q["Year"].max())
grid = (
    pd.MultiIndex.from_product([range(min_year, max_year + 1), [1, 2, 3, 4]],
                               names=["Year", "Quarter"])
      .to_frame(index=False)
      .sort_values(["Year", "Quarter"], ignore_index=True)
)
grid = grid.merge(macro_q[["Year", "Quarter"] + macro_cols],
                  on=["Year", "Quarter"], how="left")

# 3) For each Year/Quarter, take the latest *earlier* available data (ffill),
#    then shift by 1 quarter so a game in Y-Q gets macro from Y-Q-1.
prev_available = grid[macro_cols].ffill().shift(1).add_suffix("_lagQ")
lagged_q = pd.concat([grid[["Year", "Quarter"]], prev_available], axis=1)

# 4) Clean football df before merging to avoid _x/_y
RawDataB_weather = squash_suffix_pairs(RawDataB_weather)

# 5) Drop any existing lagged columns so we don't collide
lag_cols = [f"{c}_lagQ" for c in macro_cols]
RawDataB_weather.drop(columns=[c for c in lag_cols if c in RawDataB_weather.columns],
                      inplace=True, errors="ignore")

# 6) Merge by (Year, quarter) → previous quarter macro (Quarter column on right)
RawDataB_weather = (
    RawDataB_weather
      .merge(lagged_q,
             how="left",
             left_on=["Year", "quarter"],
             right_on=["Year", "Quarter"],
             suffixes=("", "_dup"))
      .drop(columns=["Quarter"] + [c for c in list(RawDataB_weather.columns) if c.endswith("_dup")],
            errors="ignore")
)

# 7) Final sweep for any residual *_x/_y from earlier steps
RawDataB_weather = squash_suffix_pairs(RawDataB_weather)

# 8) Quick sanity print
print("✅ Quarter-by-quarter macro merged (uses previous available quarter).")
print("   Added columns:", [c for c in RawDataB_weather.columns if c.endswith("_lagQ")])
print("   Rows:", len(RawDataB_weather))


In [ ]:
RawDataB_weather

### Result Handling (1)

In [ ]:
# Splitting the "Result" column into "Home Team Goals Scored" and "Away Team Goals Scored, some "score" rows contain - as they haven"t been played yet.
RawDataB_weather[["Home Team Goals Scored", "Away Team Goals Scored"]] = RawDataB_weather["Result"].apply(
    lambda x: (x.split(":") if ":" in x else ["-", "-"])
).apply(pd.Series)

In [ ]:
RawDataB_weather.drop(columns=["Result"], inplace=True)

In [ ]:
# Creating a new column "Match Type" based on conditions
def determine_match_type(value):
    if "AET" in value:
        return "Extra Time"
    elif "pens" in value:
        return "Penalties"
    else:
        return "Normal Time"

# Applying the function to create the "Match Type" column
RawDataB_weather["Match Type"] = RawDataB_weather["Away Team Goals Scored"].apply(determine_match_type)

# Optionally, remove any letters like "AET" or "on pens" from the score columns to keep only the numbers
RawDataB_weather["Away Team Goals Scored"] = RawDataB_weather["Away Team Goals Scored"].str.extract(r"(\d+)").astype(int)

In [ ]:
RawDataB_weather.head()

### Add Win/Lost

In [ ]:
# Ensure the goal columns are numeric and handle non-numeric entries (e.g., "-")
RawDataB_weather["Home Team Goals Scored"] = pd.to_numeric(RawDataB_weather["Home Team Goals Scored"], errors="coerce")
RawDataB_weather["Away Team Goals Scored"] = pd.to_numeric(RawDataB_weather["Away Team Goals Scored"], errors="coerce")

# Function to determine the match result for home and away teams
def determine_outcome(home_goals, away_goals):
    if pd.isna(home_goals) or pd.isna(away_goals):
        return "Not Played", "Not Played"  # Assign "Not Played" if either score is missing
    elif home_goals > away_goals:
        return "Win", "Loss"
    elif home_goals < away_goals:
        return "Loss", "Win"
    else:
        return "Draw", "Draw"

# Applying the function to create "Home Team Outcome" and "Away Team Outcome" columns
RawDataB_weather[["Home Team Outcome", "Away Team Outcome"]] = RawDataB_weather.apply(
    lambda row: pd.Series(determine_outcome(row["Home Team Goals Scored"], row["Away Team Goals Scored"])),
    axis=1
)

In [ ]:
RawDataB_weather.head()

### Result Handling (2) (Manual)

In [ ]:
# Match 1: Yverdon Sport vs. FC Zürich (26 October 2021)
target_row = RawData[(RawData["Weekday"] == "Tuesday") & (RawData["Month"] == 10) & (RawData["Year"] == 2021) &
                (RawData["Home Team"] == "Yverdon Sport") & (RawData["Away Team"] == "FC Zürich")]
if not target_row.empty:
    index = target_row.index[0]
    RawData.loc[index, "Home Team Goals Scored"] = 2
    RawData.loc[index, "Away Team Goals Scored"] = 2

# Match 2: Servette FC vs. FC Lugano (5 April 2023)
target_row = RawData[(RawData["Weekday"] == "Wednesday") & (RawData["Month"] == 4) & (RawData["Year"] == 2023) &
                (RawData["Home Team"] == "Servette FC") & (RawData["Away Team"] == "FC Lugano")]
if not target_row.empty:
    index = target_row.index[0]
    RawData.loc[index, "Home Team Goals Scored"] = 2
    RawData.loc[index, "Away Team Goals Scored"] = 2

# Match 3: Servette FC vs. Lausanne-Sport (1 November 2023)
target_row = RawData[(RawData["Weekday"] == "Wednesday") & (RawData["Month"] == 11) & (RawData["Year"] == 2023) &
                (RawData["Home Team"] == "Servette FC") & (RawData["Away Team"] == "Lausanne-Sport")]
if not target_row.empty:
    index = target_row.index[0]
    RawData.loc[index, "Home Team Goals Scored"] = 1
    RawData.loc[index, "Away Team Goals Scored"] = 0

# Match 4: Servette FC vs. FC Lugano (2 June 2024)
target_row = RawData[(RawData["Weekday"] == "Sunday") & (RawData["Month"] == 6) & (RawData["Year"] == 2024) &
                (RawData["Home Team"] == "Servette FC") & (RawData["Away Team"] == "FC Lugano")]
if not target_row.empty:
    index = target_row.index[0]
    RawData.loc[index, "Home Team Goals Scored"] = 0
    RawData.loc[index, "Away Team Goals Scored"] = 0

# Match 5: FC Lugano vs. FC Luzern (21 April 2022)
target_row = RawData[(RawData["Weekday"] == "Thursday") & (RawData["Month"] == 4) & (RawData["Year"] == 2022) &
                (RawData["Home Team"] == "FC Lugano") & (RawData["Away Team"] == "FC Luzern")]
if not target_row.empty:
    index = target_row.index[0]
    RawData.loc[index, "Home Team Goals Scored"] = 2
    RawData.loc[index, "Away Team Goals Scored"] = 2

# Match 6: FC Basel vs. FC Lugano (28 February 2024)
target_row = RawData[(RawData["Weekday"] == "Wednesday") & (RawData["Month"] == 2) & (RawData["Year"] == 2024) &
                (RawData["Home Team"] == "FC Basel") & (RawData["Away Team"] == "FC Lugano")]
if not target_row.empty:
    index = target_row.index[0]
    RawData.loc[index, "Home Team Goals Scored"] = 1
    RawData.loc[index, "Away Team Goals Scored"] = 1

# Match 7: FC Basel vs. Bröndby IF (11 August 2022)
target_row = RawData[(RawData["Weekday"] == "Thursday") & (RawData["Month"] == 8) & (RawData["Year"] == 2022) &
                (RawData["Home Team"] == "FC Basel") & (RawData["Away Team"] == "Bröndby IF")]
if not target_row.empty:
    index = target_row.index[0]
    RawData.loc[index, "Home Team Goals Scored"] = 2
    RawData.loc[index, "Away Team Goals Scored"] = 1

### Add nb Wins/Goals Scored/Goals conceded

In [ ]:
# 🧩 --- Parse "Date" and Prepare Rolling Stats ---

# Convert "Date" to datetime safely
RawDataB_weather["Date"] = pd.to_datetime(
    RawDataB_weather["Date"],
    errors="coerce",
    infer_datetime_format=True
)

# Extract Year, Month, and Day
RawDataB_weather["Year"] = RawDataB_weather["Date"].dt.year
RawDataB_weather["Month"] = RawDataB_weather["Date"].dt.month
RawDataB_weather["Day"] = RawDataB_weather["Date"].dt.day

# Sort by chronological order
RawDataB_weather = RawDataB_weather.sort_values(by=["Year", "Month", "Day"]).reset_index(drop=True)

# Create a binary "Win" indicator for rolling sums
RawDataB_weather["Win"] = (RawDataB_weather["Home Team Outcome"] == "Win").astype(int)

# --- Rolling stats over the last 5 home games per team ---
RawDataB_weather["Goals Scored in Last 5 Games"] = (
    RawDataB_weather.groupby("Home Team")["Home Team Goals Scored"]
    .rolling(window=5, min_periods=1).sum().shift(1).reset_index(level=0, drop=True)
)

RawDataB_weather["Goals Conceded in Last 5 Games"] = (
    RawDataB_weather.groupby("Home Team")["Away Team Goals Scored"]
    .rolling(window=5, min_periods=1).sum().shift(1).reset_index(level=0, drop=True)
)

RawDataB_weather["Number of Wins in Last 5 Games"] = (
    RawDataB_weather.groupby("Home Team")["Win"]
    .rolling(window=5, min_periods=1).sum().shift(1).reset_index(level=0, drop=True)
)

# --- Manual initialization for first 5 games per team ---
historical_data = {
    "Club Brugge": {"Goals Scored": [3, 0, 3, 1, 1], "Goals Conceded": [2, 2, 2, 0, 0], "Wins": [1, 0, 1, 1, 1]},
    "Cercle Brugge": {"Goals Scored": [0, 0, 1, 2, 2], "Goals Conceded": [4, 3, 2, 3, 6], "Wins": [0, 0, 0, 0, 0]},
    "Genk": {"Goals Scored": [0, 1, 2, 4, 1], "Goals Conceded": [0, 1, 3, 0, 0], "Wins": [0, 0, 0, 1, 1]},
    "RSC Anderlecht": {"Goals Scored": [1, 1, 1, 2, 0], "Goals Conceded": [2, 1, 1, 1, 1], "Wins": [0, 0, 0, 1, 0]},
    "Union SG": {"Goals Scored": [1, 2, 3, 2, 2], "Goals Conceded": [2, 0, 3, 2, 3], "Wins": [0, 1, 0, 1, 0]},
    "KAA Gent": {"Goals Scored": [2, 2, 3, 1, 2], "Goals Conceded": [1, 1, 2, 1, 2], "Wins": [1, 1, 1, 0, 0]},
    "Royal Antwerp": {"Goals Scored": [3, 2, 2, 1, 0], "Goals Conceded": [2, 3, 2, 1, 4], "Wins": [1, 0, 0, 1, 0]},
    "KVC Westerlo": {"Goals Scored": [1, 0, 3, 2, 1], "Goals Conceded": [2, 2, 0, 3, 0], "Wins": [0, 0, 1, 0, 1]},
    "Standard Liège": {"Goals Scored": [0, 2, 2, 2, 2], "Goals Conceded": [0, 0, 3, 2, 2], "Wins": [0, 1, 0, 0, 0]},
    "KV Mechelen": {"Goals Scored": [2, 2, 0, 1, 0], "Goals Conceded": [1, 1, 1, 1, 1], "Wins": [0, 0, 0, 0, 0]},
    "R Charleroi SC": {"Goals Scored": [2, 2, 2, 2, 3], "Goals Conceded": [3, 1, 0, 0, 0], "Wins": [0, 1, 1, 1, 1]},
    "OH Leuven": {"Goals Scored": [1, 3, 3, 2, 0], "Goals Conceded": [0, 0, 0, 2, 1], "Wins": [1, 1, 1, 0, 0]},
    "Sint-Truiden": {"Goals Scored": [2, 1, 2, 0, 2], "Goals Conceded": [1, 1, 2, 2, 2], "Wins": [1, 0, 0, 0, 0]},
    "FCV Dender EH": {"Goals Scored": [3, 0, 1, 2, 1], "Goals Conceded": [3, 2, 2, 1, 2], "Wins": [0, 0, 0, 1, 0]},
    "Zulte Waregem": {"Goals Scored": [2, 5, 0, 0, 6], "Goals Conceded": [1, 0, 5, 3, 2], "Wins": [1, 1, 0, 0, 1]},
    "La Louvière": {"Goals Scored": [7, 4, 1, 1, 0], "Goals Conceded": [0, 1, 1, 1, 2], "Wins": [1, 1, 0, 0, 0]},
}

for team, data in historical_data.items():
    mask = (RawDataB_weather["Home Team"] == team) & (RawDataB_weather.index < 5)
    RawDataB_weather.loc[mask, "Goals Scored in Last 5 Games"] = sum(data["Goals Scored"])
    RawDataB_weather.loc[mask, "Goals Conceded in Last 5 Games"] = sum(data["Goals Conceded"])
    RawDataB_weather.loc[mask, "Number of Wins in Last 5 Games"] = sum(data["Wins"])

# Replace missing or invalid values with 0 safely
RawDataB_weather = RawDataB_weather.infer_objects(copy=False).fillna(0)

# Drop the helper "Win" column
RawDataB_weather.drop(columns=["Win"], inplace=True)

# Save to new CSV file
RawDataB_weather.to_csv("Updated_Cleaned_RawDataB_weather.csv", index=False)

print("✅ Updated_Cleaned_RawDataB_weather.csv successfully saved.")

### Drop date and old date

In [ ]:
RawDataB_weather = RawDataB_weather.drop(columns=["old_date","date"])

In [ ]:
output_csv = "football_results.csv"
RawDataB_weather.to_csv(output_csv, index=False)

### Categorization of some columns

In [ ]:
RawDataB_weather["Ranking Home Team"] = pd.to_numeric(RawDataB_weather["Ranking Home Team"], errors="coerce").fillna(0).astype(int)
RawDataB_weather["Ranking Away Team"] = pd.to_numeric(RawDataB_weather["Ranking Away Team"], errors="coerce").fillna(0).astype(int)


In [ ]:
def categorize_opposing_team(ranking):
    if ranking in range(1, 4):
        return "Top ranked"
    elif ranking in range(4, 9):
        return "Medium ranked"
    elif ranking in range(9, 13):
        return "Bottom ranked"
    elif ranking == 0:
        return "Not ranked"
    else:
        return "Unknown"  # Just in case of unexpected values

RawDataB_weather["Opposing team Category"] = RawDataB_weather["Ranking Away Team"].apply(categorize_opposing_team)
RawDataB_weather["Home team Category"] = RawDataB_weather["Ranking Home Team"].apply(categorize_opposing_team)

In [ ]:
def categorize_game_day(weekday):
    if weekday in ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]:
        return "Weekday"
    elif weekday in ["Saturday", "Sunday"]:
        return "Weekend"
    else:
        return "Unknown"  # In case of unexpected values

# Applying the categorization function to create the new feature
RawDataB_weather["Game day"] = RawDataB_weather["Weekday"].apply(categorize_game_day)

In [ ]:
import re
import numpy as np

# Normalize column names (replace NBSP with normal space)
RawDataB_weather.columns = [c.replace("\xa0", " ") for c in RawDataB_weather.columns]

def parse_hour_any(s):
    if pd.isna(s):
        return np.nan
    s = str(s).strip().lower()
    # hh:mm
    m = re.match(r"^\s*(\d{1,2})\s*:\s*\d{2}\s*$", s)
    if m:
        return int(m.group(1))
    # 20.0, 20, "20h"
    m = re.match(r"^\s*(\d{1,2})(?:\.\d+)?\s*h?\s*$", s)
    if m:
        return int(m.group(1))
    # 8 pm / 8pm / 20h00
    m = re.match(r"^\s*(\d{1,2})\s*(am|pm)\s*$", s)
    if m:
        h = int(m.group(1))
        return (h % 12) if m.group(2) == "am" else (h % 12) + 12
    return np.nan

def bucket_hour(h):
    if np.isnan(h):
        return ""
    h = int(h)
    if h < 18:
        return "Afternoon"
    elif 18 <= h < 20:
        return "Evening"
    else:
        return "Night"

# Build/overwrite Time slot
hours = RawDataB_weather["Time"].apply(parse_hour_any)
RawDataB_weather["Time slot"] = hours.apply(bucket_hour)

# (Optional) sanity check
print(RawDataB_weather["Time"].head(5).tolist())
print(RawDataB_weather[["Time", "Time slot"]].head(10))


In [ ]:
unique_weather_categories = RawDataB_weather["Weather"].unique()

# Display the unique weather categories
print(unique_weather_categories)

In [ ]:
def categorize_weather(weather):
    if weather in ["Clear or mostly clear", "Partly cloudy"]:
        return "Good"
    elif weather in ["Rainy", "Drizzle", "Snowy"]:
        return "Bad"
    else:
        return "Unknown"  # In case of unexpected or incorrect values

# Applying the categorization function to create the new feature
RawDataB_weather["Weather GoodBad"] = RawDataB_weather["Weather"].apply(categorize_weather)


In [ ]:
def is_derby_boolean(home_team, away_team):
    # Jupiler Pro League derbies (teams as in your dict)
    derbies = {
        "Club Brugge": ["Cercle Brugge"],                  # Bruges Derby
        "Cercle Brugge": ["Club Brugge"],

        "RSC Anderlecht": ["Union SG"],                    # Brussels Derby
        "Union SG": ["RSC Anderlecht"],

        "Genk": ["Sint-Truiden"],                          # Limburg Derby
        "Sint-Truiden": ["Genk"],

        "Standard Liège": ["R Charleroi SC"],              # Walloon Derby
        "R Charleroi SC": ["Standard Liège"],

        "OH Leuven": ["KV Mechelen"],                      # Dijle Derby
        "KV Mechelen": ["OH Leuven"],
    }

    # Check if the home and away team form a derby
    if away_team in derbies.get(home_team, []):
        return 1
    elif home_team in derbies.get(away_team, []):
        return 1
    else:
        return 0

# Applying the categorization function to create the new feature
RawDataB_weather["Derby"] = RawDataB_weather.apply(lambda row: is_derby_boolean(row["Home Team"], row["Away Team"]), axis=1)

In [ ]:
# Manual Checking of dataset
# Save the DataFrame to a CSV file
output_csv = "CleanedData.csv"
RawDataB_weather.to_csv(output_csv, index=False)

from google.colab import files

# Download the CSV file
files.download(output_csv)

## We keep only the competition we are intrested in

---

As our objective is to predict the Jupiler pro league Games, we will keep only the rows where "competition" is one of the following:

+ Jupiler Pro League
+ Jupiler Pro League Abstiegs-Playoffs
+ Jupiler Pro League Champions' Play-Offs
+ Jupiler Pro League Europe Play-Offs
+ Jupiler Pro League Relegation Play-offs
+ Jupiler Pro League playoff Europa League

In [ ]:
# keep only JPL-related competitions
keep = [
    'Jupiler Pro League',
    'Jupiler Pro League Abstiegs-Playoffs',
    "Jupiler Pro League Champions' Play-Offs",
    'Jupiler Pro League Europe Play-Offs',
    'Jupiler Pro League Relegation Play-offs',
    'Jupiler Pro League playoff Europa League'
]

df = df[df['Competition'].isin(keep)].copy()

# drop column (now no longer informative)
df = df.drop(columns=['Competition'])
